In [33]:
import numpy as np
from scipy.optimize import fsolve

def run_task(b1, b2, C, db1_dx, db2_dx, C_dx):
	n = 2
	y_star_eq = lambda y: db1_dx(y) + db2_dx(y) - C_dx(y)
	y_star = fsolve(y_star_eq, 2)[0]
	b1_star = b1(y_star)
	b2_star = b2(y_star)
	c_star = C(y_star)
	v_1_2 = b1_star + b2_star - c_star

	v1_eq = lambda y: db1_dx(y) - C_dx(y)
	v_1 = fsolve(v1_eq, 2)[0]

	v2_eq = lambda y: db2_dx(y) - C_dx(y)
	v_2 = fsolve(v2_eq, 2)[0]

	v_delta = v_1_2 - (v_1 + v_2)

	sigma_1 = (v_1 + v_1_2 - v_2) / 2
	sigma_2 = (v_2 + v_1_2 - v_1) / 2

	x1_star = b1_star - sigma_1
	x2_star = b2_star - sigma_2

	n_core = np.min([c_star / n, np.min([b1_star, b2_star]) / n])

	partial_equilibrium_1 = lambda r: db1_dx(y_star) - r*C_dx(y_star)
	r_1 = fsolve(partial_equilibrium_1, 2)[0]

	partial_equilibrium_2 = lambda r: db2_dx(y_star) - r*C_dx(y_star)
	r_2 = fsolve(partial_equilibrium_2, 2)[0]

	x_partial_equilibrium_1 = r_1 * C(y_star)
	x_partial_equilibrium_2 = r_2 * C(y_star)

	egalitarian_eq = lambda y_h: b1(y_h) + b2(y_h) - v_1_2
	y_hat = fsolve(egalitarian_eq, 2)[0]

	x1_star_eg = b1(y_star) - b1(y_hat)
	x2_star_eg = b2(y_star) - b2(y_hat)

	return y_star, b1_star, b2_star, c_star, v_1_2, v_1, v_2, v_delta, sigma_1, sigma_2, x1_star, x2_star, n_core, r_1, r_2, x_partial_equilibrium_1, x_partial_equilibrium_2, y_hat, x1_star_eg, x2_star_eg

In [34]:
def b1(y: float) -> float:
    return 2*np.log(1 + 2*y)

def b2(y: float) -> float:
    return np.sqrt(y)

def C(y: float) -> float:
    return np.power(y,2) / 4

def db1_dx(y: float) -> float:
    return 4 / (1 + 2*y)

def db2_dx(y: float) -> float:
    return 1 / (2*np.sqrt(y))

def C_dx(y: float) -> float:
    return y / 2

run_task(b1, b2, C, db1_dx, db2_dx, C_dx)

(2.1740616459730924,
 3.3534914256316113,
 1.4744699542456239,
 1.1816360101228078,
 3.646325369754428,
 1.7655644370746375,
 1.0000000000000002,
 0.8807609326797903,
 2.2059449034145326,
 1.4403804663398954,
 1.1475465222170786,
 0.03408948790572852,
 0.5908180050614039,
 0.6880448078735384,
 0.31195519212646156,
 0.8130185215614018,
 0.36861748856140597,
 1.264370983669657,
 0.8316085771603547,
 0.35002743296245176)